In [16]:
import torch

## Creating Layers

In [2]:
class DenseLayer:
  # Layer initialization
  def __init__(self, n_inputs, n_neurons):
    # Initialize weights and biases
    self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
    self.biases = torch.zeros((1, n_neurons))

  # Forward pass
  def forward(self, inputs):
    # record the inputs
    self.inputs = inputs
    # Calculate output values from inputs, weights and biases
    self.output = torch.matmul(inputs, self.weights) + self.biases

  # Backward pass
  def backward(self, dvalues):
    # Gradients on parameters
    self.dweights = torch.dot(self.inputs.T, dvalues)
    self.dbiases = torch.sum(dvalues, axis=0, keepdims=True)
    # Gradient on values
    self.dinputs = torch.dot(dvalues, self.weights.T)

## Activation Functions

In [3]:
class Activation_ReLU:
  # Forward pass
  def forward(self, inputs):
    # Remember input values
    self.inputs = inputs
    self.output = torch.max(torch.tensor(0),inputs)
  # Backward pass
  def backward(self, dvalues):
    self.dinputs = dvalues
    # Zero gradient where input values were negative
    self.dinputs[self.inputs <= 0] = 0

In [4]:
class Activation_Softmax:
  # Forward pass
  def forward(self, inputs):
    # Get unnormalized probabilities
    exp_values = torch.exp(inputs - torch.max(inputs, axis=1, keepdim=True).values)
    # Normalize them for each sample
    probabilities = exp_values / torch.sum(exp_values, axis=1, keepdim=True)
    self.output = probabilities
  # Backward pass
  def backward(self, dvalues):
    # Create uninitialized array
    self.dinputs = torch.empty_like(dvalues)
    # Enumerate outputs and gradients
    for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
      # Flatten output array
      single_output = single_output.reshape(-1, 1)
      # Calculate Jacobian matrix of the output and
      jacobian_matrix = torch.diagflat(single_output) - torch.dot(single_output, single_output.T)
    # Calculate sample-wise gradient
    # and add it to the array of sample gradients
    self.dinputs[index] = torch.dot(jacobian_matrix, single_dvalues)

## Loss

In [5]:
class Loss_CategoricalCrossentropy() :
  # Forward pass
  def forward(self, y_pred, y_true):
    samples = len(y_pred)
    # Clip data to prevent division by 0
    # Clip both sides to not drag mean towards any value
    y_pred_clipped = torch.clip(y_pred, 1e-8, 1 - 1e-8)
    # only if categorical labels
    if len(y_true.shape) == 1:
      correct_confidences = y_pred_clipped[range(samples), y_true]
    # Mask values - only for one-hot encoded labels
    elif len(y_true.shape) == 2:
      correct_confidences = torch.sum(y_pred_clipped * y_true, axis=1)
    log_loss = -torch.log(correct_confidences)
    data_loss = torch.mean(log_loss)
    return data_loss
  
  # Backward pass
  def backward(self, dvalues, y_true):
    # Number of samples
    samples = len(dvalues)
    # Number of labels in every sample
    # We'll use the first sample to count them
    labels = len(dvalues[0])
    # If labels are sparse, turn them into one-hot vector
    if len(y_true.shape) == 1:
      y_true = torch.eye(labels)[y_true]
    # Calculate gradient
    self.dinputs = -y_true / dvalues
    # Normalize gradient
    self.dinputs = self.dinputs / samples

## Accuracy

In [6]:
class Accuracy():
  def calculate(self, y_pred, y_true):
    predictions = torch.argmax(y_pred, axis=1)
    if len(y_true.shape) == 2:
      y_true = torch.argmax(y_true, axis=1)
    accuracy = torch.mean((predictions == y_true).float())
    return accuracy

## Optimizers

In [7]:
class Optimizer_SGD:
  # Initialize optimizer - set settings,
  def __init__(self, model, learning_rate=0.01):
    self.learning_rate = learning_rate
    self.model = model
    
  def step(self):
    for layer in self.model.layers:
      self.update_params(layer)
      
  def zero_grad(self):
    for layer in self.model.layers:
      layer.dweights = 0
      layer.dbiases = 0
      
  # Update parameters
  def update_params(self, layer):
    layer.weights += -self.learning_rate * layer.dweights
    layer.biases += -self.learning_rate * layer.dbiases

## Training and Testing

In [8]:
import numpy as np
from sklearn.datasets import load_iris

In [9]:
# Load the Iris dataset from scikit-learn
iris = load_iris()
X = iris.data
y = iris.target

In [10]:
# Convert the NumPy arrays to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64)

In [11]:
print("X shape:", X.shape)
print("y shape:", y.shape)
print("Feature names:", iris.feature_names)
print("Class names:", iris.target_names)
print(X[:5])
print(y[:5])

X shape: torch.Size([150, 4])
y shape: torch.Size([150])
Feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Class names: ['setosa' 'versicolor' 'virginica']
tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [5.0000, 3.6000, 1.4000, 0.2000]])
tensor([0, 0, 0, 0, 0])


In [ ]:
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self):
        super(self).__init__()

        self.layers = [
            DenseLayer(4, 16),
            Activation_ReLU(),
            DenseLayer(16, 3),
            Activation_Softmax()
        ]

    def forward(self, x):
        x = None
        for layer in self.layers:
            x = layer.forward(x)
        return x

In [ ]:
model = CustomModel()
loss = Loss_CategoricalCrossentropy()
optimizer = Optimizer_SGD(model)

# Training loop
epochs = 100

for epoch in range(epochs):
    model.train()  # Set the model to training mode

    for batch_idx, (inputs, targets) in enumerate(torch.concat([X, y])):
        optimizer.zero_grad()  # Zero the gradients

        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        t_loss = loss.forward(outputs, targets)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        # Print progress
        if batch_idx % 100 == 0:
            print(f'Epoch {epoch + 1}/{epochs}, Batch {batch_idx}/{len(X)}, Loss: {loss.item()}')

print("Training complete!")
